# Evaluation von DialogSystemen

Ziele der heutigen Übung:
- Verschiedene quantitative Metriken eines Dialogdatensatzes berechnen
- Audio Files mittels verschiedenen Spracherkennern erkennen evaluieren

Wir arbeiten heute mit dem Datensatz der DSTC3. Die Daten sind auf folgendem Repository zu finden:

https://github.com/matthen/dstc?tab=readme-ov-file 

Lade dort die Dateien ```audio_*.tar.gz/audio_Apr11_S2.tar.gz``` und ```dstc3_test.tar.gz``` runter und entpacke diese. Wir betrachten nur Files der Unterordner Apr11_S2. Im dstc3_test findet ihr jeweils ein ```label.json``` welches den Dialog und die Transkriptionen enthält, in dem anderen Ordner findet ihr die Audiofiles.

Ladet euch zuerst die Dialogdateien aus dem label.json in eine Liste:

In [38]:
import os
import json
import pandas as pd

DATASET_FOLDER_DIALOG = "../data/DSTC/dstc3_test/data/Apr11_S2"
DATASET_FOLDER_AUDIO = "../data/DSTC/data/Apr11_S2"

dialog_list = []
for dialog_dir in os.listdir(DATASET_FOLDER_DIALOG):
    if ".DS_Store" in dialog_dir:
        continue
    with open(os.path.join(DATASET_FOLDER_DIALOG, dialog_dir, "label.json"), encoding="utf-8") as file:
        dialog_list.append(json.load(file))

## Nutzerfragebogen

Zum Ende des Tests wurde den Personen ein Fragebogen gestellt, Liste alle Fragen und die zugehörigen Antworten auf. Werte anschließen die Zufriedenheit der Versuchspersonen als Mittelwert aus. 

Hinweis: bei realen Datensätzen müssen wir immer damit rechnen, dass manche Werte nicht existieren. Überprüfe daher, ob die jeweiligen Einträge im dict auch enthalten sind.

In [39]:
def get_feedback(dialog):
    if 'task-information' in dialog:
        if 'feedback' in dialog['task-information']:
            feedback = dialog['task-information']['feedback']
            return feedback
    return None

In [ ]:
questions = []
answers = []
for dialog in dialog_list:
    feedback = get_feedback(dialog)
    if feedback and 'questionnaire' in feedback:
        for fb in feedback['questionnaire']:
            questions.append(fb[0])
            answers.append(fb[1])

#remove duplicates
print("Questions asked: ", list(set(questions)))
print("Answers given: ", list(set(answers)))


In [ ]:
# ['strongly disagree', 'disagree', 'lightly disagree', 'slightly agree', 'agree', 'strongly agree']
# [0, 1, 2, 3, 4, 5]

sum_answers = 0
for answer in answers:
    match answer:
        case 'strongly disagree':
            sum_answers += 0
        case 'disagree':
            sum_answers += 1
        case 'lightly disagree':
            sum_answers += 2
        case 'slightly agree':
            sum_answers += 3
        case 'agree':
            sum_answers += 4
        case 'strongly agree':
            sum_answers += 5

print("Number of answers: ", len(answers))
print("Average answer: : ", sum_answers / len(answers))

## Dialogmanager Metriken

Berechne die folgenden Metriken des Dialogmanagers: 
- Durchschnittliche Anzahl Nutzeräußerungen
- Task Completion

In [ ]:
succ_list = []
sum_turns = 0
for dialog in dialog_list:
    feedback = get_feedback(dialog)
    if 'success' in feedback:
        succ_list.append(feedback['success'])
    if 'turns' in dialog:
        sum_turns += len(dialog['turns'])
    
print("Task completion rate: ", succ_list.count(True)/len(succ_list))
print("Average number of user utterances: ", sum_turns/len(dialog_list))


## ASR Metriken

Wir haben Audio Files und die zugehörigen Transkriptionen im label.json. Nutze deine Implementierung mit Vosk, um den Text des jeweiligen Audio Files zu erkennen und vergleiche dies mit der Referenz. Daraus können wir nun die Word Error Rate berechnen. Mache dies erst für ein File und dann für mehrere - optimalerweise den Durchschnitt des gesamten Datensatzes, aber je nach CPU und Vosk Modell könnte das für alle etwas lange dauern ;-). Dann nehme einfach eine Teilmenge.

Um die WER zu berechnen, kann man entweder den Algorithmus der Levenshtein-Distanz nutzen oder aber die Python Bibliothek https://pypi.org/project/jiwer/.  

Vergleiche auch gerne ein kleines und ein großes Vosk Modell.

In [ ]:
pip install jiwer

In [43]:
import json
import vosk
import speech_recognition as sr

def recognize_vosk(audio_data: any, recognizer: sr.Recognizer, language = 'en') -> any:
    try:
        audio_data = recognizer.record(audio_data) 
        text = recognizer.recognize_vosk(audio_data, language=language)
        text_json = json.loads(text)
        return text_json
    except sr.RequestError:
        print("[Vosk] Error: Could not access Google Web Speech API;")
    except sr.UnknownValueError:
        print("[Vosk] Sorry, I do not understand")
    return None

In [44]:
import jiwer
from jiwer import wer

def calculate_WER(recognized: str, label: str) -> float:
    w = wer(label, recognized)
    print(f"Cmp: \t{recognized}\n\t{label}(label)\n\tWER: {w}")
    return w

In [ ]:
recognizer = sr.Recognizer()

num_utterances = 0
sum_wer = 0

for dialog in dialog_list[:10]:
    for turn in dialog['turns']:
        file = os.path.join(DATASET_FOLDER_AUDIO, dialog['session-id'], 'original_dir', turn['audio-file'])
        print(str(file))
        if os.path.exists(file):
            audio = sr.AudioFile(file)
            with audio as source:
                result = recognize_vosk(source, recognizer)
                sum_wer += calculate_WER(result['text'], turn['transcription'])
                num_utterances += 1
                #results.append({'asr': result['text']})

print(f"Average word error rate: {sum_wer/num_utterances}")